# Pipulate: SEO Investigations Using Google Sheets

## Global Module Imports & Google Services Login
These are the global imports that must remain at the top of the file and should not change.

In [ ]:
import sys, shelve, requests       # Import common libraries used throughout system
from imp import reload             # This keeps your edits to functions.ipynb in-memory
import notebook_finder             # This allows .ipynb files to be loaded like as modules
import goodsheet                   # Handles OAuth login for various Google API services
from collections import namedtuple # Allows creation of informative APIs for workflows, etc.

## Configuration Options

In [ ]:
rows_to_batch  = 20    # Number of rows to process before Sheets batch update (i.e. 10)
chunk_limit    = 0     # Limit processing to x-chunks when testing new funcs (0 = all)

## Define Interface for Creating Workflows

In [ ]:
Populate = namedtuple('Populate', 'sheet tab rows cols') # Create and populate tables
Pipulate = namedtuple('Pipulate', 'sheet tab args data cache') # Per-row lookups on tables
Python = namedtuple('Python', 'sheet function') # Arbitrary function against table

## Default Workflow
These values provide increased control and capabilities in the pipulation process.

In [ ]:
# This can be left as-is, with just the sheet & tab values changed to match your sheet.
# This can also be modified to support multi-stage workflows (to populate then pipulate)
workflow = [Pipulate(sheet="Test", tab="Sheet1", args=None, data=None, cache=False)]

## Alternative Workflow Examples

**First, we create our column names (it can alternatively be included in the list_of_lists)**

    column_names=['c1','c2','c3']

**Then, we create a list of lists**

    list_of_lists = [['1','2','3'],['a','b','c']]

**And finally, we make it an item in a workflow**

    workflow = [Populate(sheet="Test", tab="Foo", rows=list_of_lists, cols=column_names)]
    
**Multiple items can go in a workflow for more advanced multi-stage investigations**

    workflow = [Populate(sheet="Test", tab="Foo", rows=list_of_lists, cols=column_names),
                Pipulate(sheet="Test", tab="Foo", args=None, data=None, cache=False)]

In [ ]:
def pipulate(google_sheet_name, google_sheet_tab, tab_with_args=None, data=None, cache=True):
    """Update all cells in Google Sheet where columns are named with pipulate functions.
    
    The pipulate function is defines the conventional behavior of using GSheets for I/O.
    It scans from left-to-right and top-to-bottom of a Google Sheet, finding "requests".
    Requests are empty cells in columns labeled with function names found in functions module.
    Pipulate will execute the named function using values from that row as function arguments.
    The entire row's contents are fed into the named function as key/value pairs as **kwargs.
    If **kwargs proves to be too messy, decorators like @url can clean up the passed-in args.
    Results are batched-up and the sheet updated on every "rows_to_batch" number of rows.
    Because pipulate works against pre-existing rows, we must often create some rows first.
    More complex workflows can chain-up a series of row-creations and subsequent pipulations."""
    
    # Create connection to GSheets, gather important details, and fail immediately on trouble.
    print('Examining "%s" in "%s"...' % (google_sheet_tab, google_sheet_name))
    try:
        worksheet = goodsheet.connect.open(google_sheet_name).worksheet(google_sheet_tab)
        rows = worksheet.row_count
        cols = worksheet.col_count
        end_range = worksheet.get_addr_int(rows, cols)
        col_count = worksheet.col_count
    except:
        print("Error: %s" % sys.exc_info()[0])
        raise SystemExit()
    
    # Make a grab for additional key/value pair arguments (like API keys) from optional tab.
    arg_dict = {}
    arg_cells = None
    if tab_with_args:
        print('Looking for optional key/value pairs in "%s" tab...' % (tab_with_args))
        try:
            arg_sheet = goodsheet.connect.open(google_sheet_name).worksheet(tab_with_args)     
            arg_cells = arg_sheet.range("A1:B20")
        except:
            print("Error: %s" % sys.exc_info()[0])
            raise SystemExit()
    if arg_cells:
        arg_temp = None
        for an_arg in arg_cells:
            if an_arg.value:
                if an_arg.col == 1:
                    arg_temp = an_arg.value
                else:
                    if an_arg.value:
                        arg_dict[arg_temp] = an_arg.value
                    else:
                        arg_dict[arg_temp] = None
                        arg_temp = None
            else:
                break

    # Create dictionary of pipulate functions invokable using their string-names dict keys!
    pipulate_funcs = [x for x in dir(functions) if x[0] is not '_']
    func_dict = {x.lower():eval('functions.%s' % x) for x in pipulate_funcs}

    # Create list of all the values found in the 1st row so we can identify named functions.
    row1_range = 'A1:%s' % worksheet.get_addr_int(1, col_count)
    cell_range = worksheet.range(row1_range)
    col_names = [x.value.lower() for x in cell_range]

    # Find the first blank row in the sheet so we can always efficiently start in the right place.
    list_of_rows = worksheet.get_all_values() #Expensive but worth it
    first_row_with_blank = rows #Default to end-of-sheet, unless actual blanks are found.
    if len(list_of_rows) == 1:
        first_row_with_blank = 2 #If we only find 1 row (at all), then row 2 MUST be 1st blank.
    for row_dex, arow in enumerate(list_of_rows):
        for cell_dex, acell in enumerate(arow):
            if not acell and cell_range[cell_dex].value in func_dict: #Blank must be in func col.
                first_row_with_blank = row_dex+1
                break
        else:
            continue
        break
    list_of_rows = None #Blank no longer necessary potentially large object (be kind to memory)
    
    # Cunk the sheet into a series of ranges that will be used for batch-updates
    chunk_ranges = [(x+1, x+rows_to_batch+1) for x in list(range(rows-1)) if x%rows_to_batch == 0]
    chunks_to_go = [(x,y) for x,y in chunk_ranges if y > first_row_with_blank]
    
    # Pipulate each chunk
    for chunk_dex, (row_start, row_end) in enumerate(chunks_to_go):
        if chunk_limit > 0 and chunk_dex >= chunk_limit:
            break
        
        # Convert each chunk into Excel-like A2:B2 range notation
        top_left = worksheet.get_addr_int(row_start+1, 1)
        lower_right = worksheet.get_addr_int(row_end, cols)
        range_string = "%s:%s" % (top_left, lower_right)
        
        # Calculate correct number of rows in the last (likely) uneven chunk
        if chunk_dex+1 == len(chunk_ranges):
            range_string = "%s:%s" % (top_left, end_range)
        print("Pipulating range %s of %s (%s)" % (chunk_dex+1, len(chunks_to_go), range_string))
        print("%s updating in: " % range_string, end="")

        # Use chunk range to create new GSpread worksheet obj for writing-back response values.
        try:
            chunk_range = worksheet.range(range_string)
        except:
            print("Error: %s" % sys.exc_info()[0])
            break
            
        # Step through each cell in the current chunk.
        row_dict = {} #Will contain all values from row and be passed to function as **kwargs
        count_down = int(len(chunk_range)/col_count)+1
        for cell_dex, acell in enumerate(chunk_range): #Working with cells, but interested in rows
            row, col, val = acell.row, acell.col, acell.value
            # Now we pipulate this row's row_dict with column-name/cell-value pairs
            row_dict[col_names[col-1]] = val
            if col%col_count == 0: #This is how we determine reaching last cell of row
                count_down = count_down - 1
                print("%s, " % count_down, end="")
                requests_response = None #HTML-cache object loop-leak prevention
                Response = None #Function-return loop-leak prevention
                # Why fetch the HTML for a URL more than once, if you don't have to?
                if 'url' in row_dict:
                    with shelve.open('urls') as urls:
                        if cache == True and row_dict['url'] in urls.keys():
                            requests_response = urls[row_dict['url']]
                        else:
                            try:
                                requests_response = requests.get(row_dict['url'])
                                urls[row_dict['url']] = requests_response #The moment of pickling
                            except requests.exceptions.RequestException as e:
                                print("(bad url) ", end="")
                                continue
                        # We now make the ENTIRE response object available to pipulate functions.
                        row_dict['response'] = requests_response
                # If data exists, we make it mutably available (for memory) to every row
                if data:
                    row_dict['data'] = data
                for key, val in row_dict.items():
                    if not val: #Only process empty cells in function-named columns
                        if key in [x.lower() for x in dir(functions) if x[0] is not '_']:
                            if arg_dict:
                                row_dict = {**arg_dict, **row_dict}
                                if '' in row_dict:
                                    del row_dict['']
                                if 'response' in row_dict:
                                    del row_dict['response']
                            try:
                                Response = func_dict[key](**row_dict) #pipulate!
                            except:
                                print("%s error in %s." % (sys.exc_info()[0], key))
                                continue
                            # We pipulate at end-of-row but update back to earlier cells from row.
                            row_start = cell_dex-cols+1
                            func_dex = col_names.index(key)
                            update_cell = row_start + func_dex
                            if Response.ok:
                                chunk_range[update_cell].value = Response.text #uncommitted
                            else:
                                chunk_range[update_cell].value = "Err: %s" % Response.status_code
                            row_dict = {} # Blank the now-used row_dict just for good measure
        try:
            # Batch update Google Sheets with the modified chunk_range.
            worksheet.update_cells(chunk_range)
            print("Range updated!")
        except:
            print(bad_api_message)
    print("Pipulation complete!") #do a little dance
    worksheet = None

In [ ]:
def populate(sheet, tab, rows, cols=None):
    print('Checking for "%s" in "%s"...' % (tab, sheet))
    try:
        spread = goodsheet.connect.open(sheet)
        list_of_sheets = spread.worksheets()
    except:
        print("Error: %s" % sys.exc_info()[0])
        raise SystemExit()
    sheets = [x.title for x in list_of_sheets]
    list_of_lists = rows
    if cols and len(cols) == len(rows[0]):
        headers = [cols]
        list_of_lists = headers + rows
    numrows = len(list_of_lists)
    numcols = len(list_of_lists[0])
    try:
        if tab not in sheets:
            print("Creating %s" % tab)
            spread.add_worksheet(tab, numcols, numrows)
            worksheet = goodsheet.connect.open(sheet).worksheet(tab)
        else:
            print("Resetting %s" % tab)
            worksheet = goodsheet.connect.open(sheet).worksheet(tab)
            worksheet.resize(rows=numrows, cols=numcols)
        end_range = worksheet.get_addr_int(numrows, numcols)
    except:
        print("Error: %s" % sys.exc_info()[0])
        raise SystemExit()            
    range_string = "A1:%s" % end_range
    chunk_range = worksheet.range(range_string)
    flat_list = [item for sublist in list_of_lists for item in sublist]
    for index, a_cell in enumerate(chunk_range):
        a_cell.value = flat_list[index]
    try:
        worksheet.update_cells(chunk_range)
    except:
        print("Error: %s" % sys.exc_info()[0])
        raise SystemExit()            

In [ ]:
if __name__ == '__main__':
    import functions
    reload(functions)
    import functions #2nd import intentional (for notebook_finder nuances)
    print("You are logged in as %s" % goodsheet.get_email())
    
    # All errors get this message, so check filename & tabname carefully & read tracebacks
    bad_api_message = "Can't reach Spreadsheet or nothing to pipulate. Double-check & try again."
    
    print("Workflow beginning.")
    for a_tuple in workflow:
        tuple_name = type(a_tuple).__name__
        if tuple_name == 'Pipulate':
            print("Pipulate request being processed...")
            sheet, tab, args, data, cache = a_tuple
            pipulate(sheet, tab, args, data, cache)
        elif tuple_name == 'Populate':
            print("Populate request being processed...")
            sheet, tab, rows, cols = a_tuple
            populate(sheet, tab, rows, cols)
    print("Workflow complete.")


In [ ]:
functions.blah()